In [203]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import os
# os.chdir("../")


In [204]:
data=pd.read_csv("../merged/len3_ship_agg_drop_never.csv",encoding="shift-jis",index_col=0)
price_data=pd.read_csv("../merged/len3_ndb_mean_prices.csv",encoding="shift-jis",index_col=0)
price_data["薬効分類"]=price_data["薬効分類"].astype(int)
price_data.rename({"薬効分類":"id"},axis=1,inplace=True)
price_data

,year,id,mean_price,mean_price_g,mean_price_b
0,2014,114,31.915396,7.283263,39.530262
1,2014,121,6.583267,5.665076,6.644077
2,2014,123,12.739638,6.337716,13.688958
3,2014,131,159.247093,93.627440,169.652616
4,2014,132,212.212151,469.346625,205.236228
...,...,...,...,...,...
195,2021,441,7.415946,5.700000,8.310868
196,2021,449,38.479161,23.000160,62.492515
197,2021,625,1048.324019,108.788878,1997.706250
198,2021,629,304.274861,79.016934,393.776286


In [205]:
data

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_264,id_265,id_332,id_339,id_399,id_441,id_449,id_625,id_629,id_799
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,2008,121,局所麻酔剤,12649530,0.0,0,1676687.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
2,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
3,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021,441,抗ヒスタミン剤,2350576,1798389.0,470949,724732.0,588014.0,209582.0,929665.0,...,0,0,0,0,0,1,0,0,0,0
346,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,0,0,0,0,1,0,0,0
347,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,0,0,0,0,1,0,0
348,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,...,0,0,0,0,0,0,0,0,1,0


In [206]:
data=data.merge(price_data,how="left",on=["year","id"])
data["nonself_dom"]=data["otc_dom"]-data["self_dom"]
data[data["mean_price"].isnull()]


,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_399,id_441,id_449,id_625,id_629,id_799,mean_price,mean_price_g,mean_price_b,nonself_dom
0,2008,114,解熱鎮痛消炎剤,113733431,32062003.0,0,16419098.0,1829448.0,0.0,NaN,...,0,0,0,0,0,0,NaN,NaN,NaN,32062003.0
1,2008,121,局所麻酔剤,12649530,0.0,0,1676687.0,0.0,0.0,NaN,...,0,0,0,0,0,0,NaN,NaN,NaN,0.0
2,2008,123,自律神経剤,5180925,0.0,0,338320.0,0.0,0.0,NaN,...,0,0,0,0,0,0,NaN,NaN,NaN,0.0
3,2008,131,眼科用剤,200817924,43657879.0,0,27228595.0,6132288.0,0.0,NaN,...,0,0,0,0,0,0,NaN,NaN,NaN,43657879.0
4,2008,132,耳鼻科用剤,23901015,15040127.0,0,3550000.0,3634742.0,0.0,NaN,...,0,0,0,0,0,0,NaN,NaN,NaN,15040127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2013,441,抗ヒスタミン剤,4385031,1252178.0,0,689878.0,129730.0,0.0,686598.0,...,0,1,0,0,0,0,NaN,NaN,NaN,1252178.0
146,2013,449,その他のアレルギー用薬,336784001,2359049.0,0,38304642.0,121849.0,0.0,29463504.0,...,0,0,1,0,0,0,NaN,NaN,NaN,2359049.0
147,2013,625,抗ウイルス剤,173656918,553900.0,0,72996273.0,173362.0,0.0,56113294.0,...,0,0,0,1,0,0,NaN,NaN,NaN,553900.0
148,2013,629,その他の化学療法剤,31490106,0.0,0,3643031.0,0.0,0.0,4754644.0,...,0,0,0,0,1,0,NaN,NaN,NaN,0.0


In [207]:
data=data[data["mean_price"].notnull()]
data

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_399,id_441,id_449,id_625,id_629,id_799,mean_price,mean_price_g,mean_price_b,nonself_dom
150,2014,114,解熱鎮痛消炎剤,140580346,48217334.0,0,32986902.0,4489227.0,0.0,31423281.0,...,0,0,0,0,0,0,31.915396,7.283263,39.530262,48217334.0
151,2014,121,局所麻酔剤,12343814,0.0,0,1340818.0,0.0,0.0,693620.0,...,0,0,0,0,0,0,6.583267,5.665076,6.644077,0.0
152,2014,123,自律神経剤,2463354,0.0,0,181986.0,0.0,0.0,121496.0,...,0,0,0,0,0,0,12.739638,6.337716,13.688958,0.0
153,2014,131,眼科用剤,286216591,59438256.0,0,24420274.0,13291858.0,0.0,29997089.0,...,0,0,0,0,0,0,159.247093,93.627440,169.652616,59438256.0
154,2014,132,耳鼻科用剤,40223232,19292767.0,0,4853936.0,5954843.0,0.0,2038010.0,...,0,0,0,0,0,0,212.212151,469.346625,205.236228,19292767.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021,441,抗ヒスタミン剤,2350576,1798389.0,470949,724732.0,588014.0,209582.0,929665.0,...,0,1,0,0,0,0,7.415946,5.700000,8.310868,1327440.0
346,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,0,1,0,0,0,38.479161,23.000160,62.492515,98151.0
347,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,0,0,1,0,0,1048.324019,108.788878,1997.706250,0.0
348,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,...,0,0,0,0,1,0,304.274861,79.016934,393.776286,0.0


In [208]:
data_2020=data.copy()
data_2020.reset_index(drop=True,inplace=True)
data_2020=data_2020[data_2020["year"]==2020]
data_2020

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_399,id_441,id_449,id_625,id_629,id_799,mean_price,mean_price_g,mean_price_b,nonself_dom
150,2020,114,解熱鎮痛消炎剤,121722673,51605038.0,36773094,38680620.0,10924975.0,8316947.0,37427896.0,...,0,0,0,0,0,0,22.542793,11.945934,35.425163,14831944.0
151,2020,121,局所麻酔剤,11089368,0.0,0,3173594.0,0.0,0.0,2721089.0,...,0,0,0,0,0,0,34.224085,5.762740,35.816261,0.0
152,2020,123,自律神経剤,1986451,0.0,0,481555.0,0.0,0.0,443724.0,...,0,0,0,0,0,0,10.910605,5.902479,12.046935,0.0
153,2020,131,眼科用剤,292043906,63783243.0,3624735,52376923.0,15484768.0,2332721.0,35943525.0,...,0,0,0,0,0,0,193.749033,112.463539,214.934762,60158508.0
154,2020,132,耳鼻科用剤,31787142,18464374.0,3023663,20279338.0,6625162.0,2037660.0,17291231.0,...,0,0,0,0,0,0,207.602785,631.301948,174.922941,15440711.0
155,2020,218,高脂血症用剤,218674039,552752.0,124161,40927572.0,82028.0,31021.0,40129196.0,...,0,0,0,0,0,0,44.411350,21.900052,106.652040,428591.0
156,2020,223,去たん剤,27423364,43640.0,43640,8530676.0,5813.0,5813.0,7978099.0,...,0,0,0,0,0,0,14.115946,11.411303,26.815248,0.0
157,2020,225,気管支拡張剤,44782657,0.0,0,26488054.0,0.0,0.0,14658061.0,...,0,0,0,0,0,0,108.970632,18.598614,191.745178,0.0
158,2020,229,その他の呼吸器官用薬,134527734,1442023.0,0,60469685.0,215616.0,0.0,29964969.0,...,0,0,0,0,0,0,2221.647343,609.999733,2549.911196,1442023.0
159,2020,231,止しゃ剤，整腸剤,18857771,7300280.0,316033,5050215.0,4601645.0,180219.0,4760184.0,...,0,0,0,0,0,0,12.164136,6.631021,19.428615,6984247.0


\begin{align*}
 N&: \mathrm{OTC}の1パッケージに含まれる量、N日分\\
 p_{i,t}&: \mathrm{OTC}の1日分あたりの価格\\
 薬価_{i,t}&:薬効分類i,t年の平均薬価
\end{align*}
予想、消費者の直面する支払額は同じ
\begin{align*}
 \frac{3}{10}(薬価_{i,t}+医療費)\times N &\simeq \hat p_{i,t} \times N\\
\alpha 薬価_{i,t}&\simeq \hat p_{i,t}
\end{align*}
次を満たす$ \alpha, N$を探す
\begin{align*}
\frac{\mathrm{Sales}_{i,t}}{\hat p_{i,t}\times N}&=\hat q_{i,t}\\
\frac{\mathrm{Sales}_{t}}{\sum_i \hat q_{i,t}}&=\bar p \\
\frac{\mathrm{Sales}_{t}}{\sum_i \mathrm{Sales}_{i,t}/(\alpha 薬価_{i,t} N)}&=\bar p 
\end{align*}

In [209]:
import numpy as np
from scipy.optimize import minimize
# def calc_mean_price(alpha,N,column,target=1234):
mean_price=0

def calc_mean_price(N,column,target=1234,k=0.7):
    # data_2020[f"{column}_packages"]=data_2020[f"{column}_dom"]*1000/(data_2020["mean_price"]*alpha*N)  
    data_2020[f"{column}_packages"]=data_2020[f"{column}_dom"]*1000/(k*0.3*(data_2020["mean_price_g"]*N+5400))
    mean_price=data_2020[f"{column}_dom"].sum()*1000/data_2020[f"{column}_packages"].sum()
    # print(mean_price)
    return abs(mean_price-target)
# 初期推定値
initial_guess = 10  # 任意の初期値を設定

# 制約条件
# constraints = ({'type': 'ineq', 'fun': lambda params: params[0] - 1})  # alpha >= 1

# 最適化を実行
result= minimize(calc_mean_price, initial_guess,args=("self",1234), method='L-BFGS-B', bounds=[(1, None)])
# N=20
# result = minimize(calc_mean_price, initial_guess,args=("self",1234), method='BFGS')
optimal_alpha = 0
N = result.x
# print(result)
# 結果を出力
print(f"Optimal alpha: {optimal_alpha}")
print(f"Optimal N: {N}")
column="self"
print(f"Optimal mean_price: ",data_2020[f"{column}_dom"].sum()*1000/data_2020[f"{column}_packages"].sum(),data_2020[f"{column}_packages"].sum()*N)
result = minimize(calc_mean_price, initial_guess,args=("nonself",994,0.55),  method='L-BFGS-B', bounds=[(1, None)])
# print(result)
# 最適なalphaとNの値
N  = result.x

# 結果を出力
print(f"Optimal alpha: {optimal_alpha}")
print(f"Optimal N: {N}")
column="nonself"
print(f"Optimal mean_price: ",data_2020[f"{column}_dom"].sum()*1000/data_2020[f"{column}_packages"].sum())
# result

Optimal alpha: 0
Optimal N: [14.22137785]
Optimal mean_price:  1234.0000000374775 [1.15695774e+09]
Optimal alpha: 0
Optimal N: [6.91868576]
Optimal mean_price:  994.0000000665398


In [201]:
# 最適化を実行
# qs=[]
# for i in range(60,85,5):
#     k=i/100
#     print(k)
#     result= minimize(calc_mean_price, initial_guess,args=("self",1234,k), method='L-BFGS-B', bounds=[(1, None)])
#     # N=20
#     # result = minimize(calc_mean_price, initial_guess,args=("self",1234), method='BFGS')
#     optimal_alpha = 0
#     N = result.x[0]
#     # print(result)
#     # 結果を出力
#     # print(f"Optimal alpha: {optimal_alpha}")
#     print(f"Optimal N: {N}")
#     column="self"
#     print(f"Optimal mean_price: ",data_2020[f"{column}_dom"].sum()*1000/data_2020[f"{column}_packages"].sum(),data_2020[f"{column}_packages"].sum()*N)
#     qs.append(data_2020[f"{column}_packages"].sum()*N)
# np.array(qs)/qs[-1]
# np.array(qs)


0.6
Optimal N: 45.64896114272092
Optimal mean_price:  1234.0000000395582 3713699171.9158063
0.65
Optimal N: 27.198133136346545
Optimal mean_price:  1234.0000000354287 2212661186.103469
0.7
Optimal N: 12.761688789571231
Optimal mean_price:  1234.0000000670998 1038207045.7385937
0.75
Optimal N: 1.9848686359651269
Optimal mean_price:  1234.00000004597 161475854.54707426
0.8
Optimal N: 1.0
Optimal mean_price:  1306.4307728361835 76843047.55165789


array([3.71369917e+09, 2.21266119e+09, 1.03820705e+09, 1.61475855e+08,
       7.68430476e+07])

In [202]:
N=14
alpha=[3.7,2.14] 
for i,column in enumerate(["self","nonself"]):
    data[f"{column}_packages"]=data[f"{column}_dom"]/(data["mean_price"]*alpha[i]*N)
    print(data[f"{column}_packages"].sum()*N)

8209483.314845356
18629952.904240478


In [57]:
2.14/3.7

0.5783783783783784

In [58]:
# data=data_2020
data["nonself_quantity"]=data["nonself_packages"]*N
data["self_quantity"]=data["self_packages"]*N
data["otc_quantity"]=data["self_quantity"]+data["nonself_quantity"]

sum(data["self_dom"])/sum(data["self_quantity"])*N,sum(data["nonself_dom"])/sum(data["nonself_quantity"])*N,sum(data["otc_dom"])/sum(data["otc_quantity"])*N

(1340.7477450007266, 1107.0751963793462, 1178.5495326279026)

In [59]:
data

,year,id,efficacy,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,stock_rx_lag,...,id_799,mean_price,mean_price_g,mean_price_b,nonself_dom,self_packages,nonself_packages,nonself_quantity,self_quantity,otc_quantity
150,2014,114,解熱鎮痛消炎剤,140580346,48217334.0,0,32986902.0,4489227.0,0.0,31423281.0,...,0,31.915396,7.283263,39.530262,48217334.0,0.000000,50426.770113,705974.781576,0.000000,705974.781576
151,2014,121,局所麻酔剤,12343814,0.0,0,1340818.0,0.0,0.0,693620.0,...,0,6.583267,5.665076,6.644077,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
152,2014,123,自律神経剤,2463354,0.0,0,181986.0,0.0,0.0,121496.0,...,0,12.739638,6.337716,13.688958,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
153,2014,131,眼科用剤,286216591,59438256.0,0,24420274.0,13291858.0,0.0,29997089.0,...,0,159.247093,93.627440,169.652616,59438256.0,0.000000,12458.126486,174413.770799,0.000000,174413.770799
154,2014,132,耳鼻科用剤,40223232,19292767.0,0,4853936.0,5954843.0,0.0,2038010.0,...,0,212.212151,469.346625,205.236228,19292767.0,0.000000,3034.467303,42482.542241,0.000000,42482.542241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,2021,441,抗ヒスタミン剤,2350576,1798389.0,470949,724732.0,588014.0,209582.0,929665.0,...,0,7.415946,5.700000,8.310868,1327440.0,1225.963497,5974.568037,83643.952511,17163.488958,100807.441469
346,2021,449,その他のアレルギー用薬,226049764,7060159.0,6962008,51409796.0,4606525.0,4528553.0,60509613.0,...,0,38.479161,23.000160,62.492515,98151.0,3492.843809,85.138761,1191.942658,48899.813328,50091.755986
347,2021,625,抗ウイルス剤,215855691,364231.0,364231,185416723.0,227307.0,227307.0,143728443.0,...,0,1048.324019,108.788878,1997.706250,0.0,6.707360,0.000000,0.000000,93.903039,93.903039
348,2021,629,その他の化学療法剤,56083192,0.0,0,12140098.0,0.0,0.0,18205617.0,...,0,304.274861,79.016934,393.776286,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
data.to_csv("../merged/len3_ship_agg_drop_never_blp.csv",encoding="shift-jis")